In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# List to store DataFrames
dfs = []
# Walk through the directory and read CSV files
for dirname, _, filenames in os.walk('C:\\Users\\jisha\\NetworkIntrusion'):
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirname, filename)
            try:
                df = pd.read_csv(file_path)
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

In [3]:
# Data dimensions of each individual dataset
for i, data in enumerate(dfs, start=1):
    rows, cols = data.shape
    print(f'df{i} -> {rows} rows, {cols} columns')

df1 -> 225745 rows, 79 columns
df2 -> 286467 rows, 79 columns
df3 -> 191033 rows, 79 columns
df4 -> 529918 rows, 79 columns
df5 -> 288602 rows, 79 columns
df6 -> 170366 rows, 79 columns
df7 -> 445909 rows, 79 columns
df8 -> 692703 rows, 79 columns


In [4]:
# Concatenate all DataFrames into a single DataFrame
data = pd.concat(dfs, axis=0, ignore_index=True)

# Deleting DataFrames after merging
for df in dfs: del df

In [5]:
# Display the first few rows
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
# Get dataset dimensions
print(f"Dataset Dimensions: {data.shape}")

Dataset Dimensions: (2830743, 79)


In [7]:
# Display data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [8]:
# Checking for missing values
missing_values = data.isna().sum()
missing_percentage = (missing_values / len(data)) * 100

# Printing columns with missing values
for column, count in missing_values.items():
    if count != 0:
        print(f"Column '{column}' has {count} missing values, which is {missing_percentage[column]:.2f}% of the total")

Column 'Flow Bytes/s' has 1358 missing values, which is 0.05% of the total


In [9]:
# Removal of leading/trailing whitespace
col_names = {col: col.strip() for col in data.columns}
data.rename(columns = col_names, inplace = True)

In [10]:
# Checking and counting duplicates
duplicates = data.duplicated()
duplicate_count = duplicates.sum()

# Output results
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 308381


In [11]:
# Removal of duplicates
data = data.drop_duplicates(keep='first')
del duplicates
data.shape

(2522362, 79)

In [12]:
# Identify columns with identical data
identical_columns = {}
columns = data.columns
list_control = columns.copy().tolist()

# Compare each pair of columns
for col1 in columns:
    for col2 in columns:
        if col1 != col2:
            if data[col1].equals(data[col2]):
                if (col1 not in identical_columns) and (col1 in list_control):
                    identical_columns[col1] = [col2]
                    list_control.remove(col2)
                elif (col1 in identical_columns) and (col1 in list_control):
                    identical_columns[col1].append(col2)
                    list_control.remove(col2)
# Print the result
if identical_columns:
    print("Identical columns found:")
    for key, value in identical_columns.items():
        print(f"'{key}' is identical to {value}")
else: print("No identical columns found.")

Identical columns found:
'Total Fwd Packets' is identical to ['Subflow Fwd Packets']
'Total Backward Packets' is identical to ['Subflow Bwd Packets']
'Fwd PSH Flags' is identical to ['SYN Flag Count']
'Bwd PSH Flags' is identical to ['Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
'Fwd URG Flags' is identical to ['CWE Flag Count']
'Fwd Header Length' is identical to ['Fwd Header Length.1']


In [13]:
# Removing the columns with duplicated values
for key, value in identical_columns.items():
    data.drop(columns=value, inplace=True)

In [14]:
print(data.columns)
data.shape

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'RST Flag Count',
       'PSH Flag C

(2522362, 67)

In [15]:
# Checking for infinite values
num_columns = data.select_dtypes(include = np.number).columns
has_infinite = np.isinf(data[num_columns]).sum()
print(has_infinite[has_infinite > 0])

# Treating infinite values
data.replace([np.inf, -np.inf], np.nan, inplace=True)

Flow Bytes/s      1211
Flow Packets/s    1564
dtype: int64


In [16]:
row_missing_percentage = (data.isna().sum(axis=1) / data.shape[1]) * 100
print(row_missing_percentage.describe())

missing_rows = data.isna().any(axis=1).sum()
print(f'\nTotal rows with missing values: {missing_rows}')

# Dropping missing values
data = data.dropna()
print(f'Dataset shape after row-wise removal: {data.shape}')

count    2.522362e+06
mean     1.850907e-03
std      7.430795e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.985075e+00
dtype: float64

Total rows with missing values: 1564
Dataset shape after row-wise removal: (2520798, 67)


In [17]:
# Mapping the attacks to the new group
group_mapping = {
    'BENIGN': 'Normal Traffic',
    'DoS Hulk': 'DoS',
    'DDoS': 'DDoS',
    'PortScan': 'Port Scanning',
    'DoS GoldenEye': 'DoS',
    'FTP-Patator': 'Brute Force',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'SSH-Patator': 'Brute Force',
    'Bot': 'Bots',
    'Web Attack � Brute Force': 'Web Attacks',
    'Web Attack � XSS': 'Web Attacks',
    'Infiltration': 'Infiltration',
    'Web Attack � Sql Injection': 'Web Attacks',
    'Heartbleed': 'Miscellaneous'
    }

# Map to new group column
data['Attack Type'] = data['Label'].map(group_mapping)

data['Attack Type'].value_counts()

Attack Type
Normal Traffic    2095057
DoS                193745
DDoS               128014
Port Scanning       90694
Brute Force          9150
Web Attacks          2143
Bots                 1948
Infiltration           36
Miscellaneous          11
Name: count, dtype: int64

In [18]:
# Dropping the old 'Label' column
data.drop(columns='Label', inplace=True)

In [19]:
data.sample(10)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Attack Type
1570151,53,511,2,2,72,246,36,36,36.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
674448,443,5382130,8,6,390,3716,209,0,48.750000,77.135780,...,20,137181.0,0.0,137181,137181,5244946.0,0.0,5244946,5244946,Normal Traffic
1488484,443,78983,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
2683056,53,24425,2,2,78,134,39,39,39.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
1411494,53,30934,2,2,90,122,45,45,45.000000,0.000000,...,36,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
1099057,53,186,2,2,96,180,48,48,48.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
734609,443,7273323,9,7,986,3244,373,0,109.555556,153.566037,...,20,1485238.0,0.0,1485238,1485238,5788037.0,0.0,5788037,5788037,Normal Traffic
521360,22,1465522,35,40,2686,6718,408,0,76.742857,105.426959,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
2673514,59684,25,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Normal Traffic
2327775,80,116263681,10,7,312,11595,312,0,31.200000,98.663063,...,32,987.0,0.0,987,987,58100000.0,59500000.0,100000000,16000000,DoS


In [20]:
print(data.shape)           # Rows, columns
print(data['Attack Type'].value_counts())  # Class distribution


(2520798, 67)
Attack Type
Normal Traffic    2095057
DoS                193745
DDoS               128014
Port Scanning       90694
Brute Force          9150
Web Attacks          2143
Bots                 1948
Infiltration           36
Miscellaneous          11
Name: count, dtype: int64


In [21]:
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Sample 100,000 rows for faster training/testing
data_sample = data.sample(100_000, random_state=42).copy()



In [22]:
# Merge rare classes into 'Other'
data_sample['Attack Type'] = data_sample['Attack Type'].replace(['Infiltration', 'Miscellaneous'], 'Other')

# Encode labels
le = LabelEncoder()
y = le.fit_transform(data_sample['Attack Type'])



In [23]:
# Drop label column for features
X = data_sample.drop('Attack Type', axis=1)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance
sample_weights = compute_sample_weight(class_weight='balanced', y=y)

# Train-test split
X_train, X_test, y_train, y_test, sw_train, sw_test = train_test_split(
    X_scaled, y, sample_weights, test_size=0.2, random_state=42, stratify=y
)


In [24]:

# Simpler base learners for speed
base_learners = [
    ('rf', RandomForestClassifier(
        n_estimators=50, max_depth=10, class_weight='balanced', random_state=42)),
    ('dt', DecisionTreeClassifier(
        max_depth=5, class_weight='balanced', random_state=42)),
    ('gb', GradientBoostingClassifier(
        n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42)),
    ('et', ExtraTreesClassifier(
        n_estimators=100, max_depth=15, class_weight='balanced', random_state=42))
]

# Lightweight meta-classifier
meta_model = LogisticRegression(max_iter=100, class_weight='balanced')


In [25]:

# Stacking classifier with passthrough and reduced CV
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_model,
    passthrough=False,
    n_jobs=-1,
    cv=5
)


In [26]:

# Train
print("Training stacking classifier on sample")
stacking_clf.fit(X_train, y_train)
print("Finished training.")


Training stacking classifier on sample...
Finished training.


In [27]:

# Evaluate
y_pred = stacking_clf.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Classification Report:
                precision    recall  f1-score   support

          Bots       0.05      0.94      0.10        16
   Brute Force       0.91      1.00      0.95        68
          DDoS       1.00      1.00      1.00      1004
           DoS       0.99      0.99      0.99      1531
Normal Traffic       1.00      0.97      0.99     16639
 Port Scanning       0.99      1.00      0.99       726
   Web Attacks       0.08      0.88      0.15        16

      accuracy                           0.98     20000
     macro avg       0.72      0.97      0.74     20000
  weighted avg       1.00      0.98      0.99     20000


Confusion Matrix:
[[   15     0     0     0     1     0     0]
 [    0    68     0     0     0     0     0]
 [    0     0  1003     0     0     0     1]
 [    0     0     0  1521     0     0    10]
 [  261     7     1    11 16200     9   150]
 [    0     0     0     1     0   725     0]
 [    0     0     0     0     2     0    14]]


In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("🔍 Detailed Evaluation Metrics:\n")

# Overall accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# Weighted scores (handle class imbalance)
print(f"Precision (weighted): {precision_score(y_test, y_pred, average='weighted'):.4f}")
print(f"Recall (weighted): {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1 Score (weighted): {f1_score(y_test, y_pred, average='weighted'):.4f}")


🔍 Detailed Evaluation Metrics:

Accuracy: 0.9773
Precision (weighted): 0.9969
Recall (weighted): 0.9773
F1 Score (weighted): 0.9864
